# Hearthstone Project
***
***

# Setup
***

In [4]:
import pandas as pd

# Acquire
***

In [65]:
df = pd.read_csv('hearthstone_standard_cards.csv')

In [66]:
df.head()

,id,collectible,slug,classId,multiClassIds,cardTypeId,cardSetId,rarityId,artistName,manaCost,...,flavorText,cropImage,duels,minionTypeId,health,attack,keywordIds,childIds,durability,armor
0,58607,1,58607-blur,14,[],5,1463,1,Zoltan Boros,0,...,"""What are you trying to tell me? That I can do...",https://d15f34w2p8l1cc.cloudfront.net/hearthst...,"{'relevant': True, 'constructed': True}",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,56806,1,56806-shadowhoof-slayer,14,[],4,2,2,A.J. Nazzaro,1,...,Was a humble Elven Archer before Sargeras got ...,https://d15f34w2p8l1cc.cloudfront.net/hearthst...,"{'relevant': True, 'constructed': True}",15.0,1.0,2.0,[8],NaN,NaN,NaN
2,58170,1,58170-crimson-sigil-runner,14,[],4,1414,1,Arthur Bozonnet,1,...,"He's not the fastest of the bunch, but he'll t...",https://d15f34w2p8l1cc.cloudfront.net/hearthst...,NaN,NaN,1.0,1.0,[86],NaN,NaN,NaN
3,59394,1,59394-demon-companion,14,"[14, 3]",5,1443,3,Zoltan Boros,1,...,Owning a pet can be real hell at times.,https://d15f34w2p8l1cc.cloudfront.net/hearthst...,"{'relevant': True, 'constructed': True}",NaN,NaN,NaN,NaN,"[59391, 59392, 59393]",NaN,NaN
4,59606,1,59606-double-jump,14,[],5,1443,1,A.J. Nazzaro,1,...,The key to every speed run.,https://d15f34w2p8l1cc.cloudfront.net/hearthst...,"{'relevant': True, 'constructed': True}",NaN,NaN,NaN,[86],NaN,NaN,NaN


# Prepare
***

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1289 entries, 0 to 1288
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1289 non-null   int64  
 1   collectible    1289 non-null   int64  
 2   slug           1289 non-null   object 
 3   classId        1289 non-null   int64  
 4   multiClassIds  1289 non-null   object 
 5   cardTypeId     1289 non-null   int64  
 6   cardSetId      1289 non-null   int64  
 7   rarityId       1289 non-null   int64  
 8   artistName     1288 non-null   object 
 9   manaCost       1289 non-null   int64  
 10  name           1289 non-null   object 
 11  text           1271 non-null   object 
 12  image          1289 non-null   object 
 13  imageGold      805 non-null    object 
 14  flavorText     1289 non-null   object 
 15  cropImage      1289 non-null   object 
 16  duels          708 non-null    object 
 17  minionTypeId   321 non-null    float64
 18  health  

- data types are acceptable for expected operations
- many null values that need to be addressed
    - artist name
    - text
    - image gold
    - duels
    - minion type id
    - health
    - attack
    - keyword ids
    - child ids
    - durability
    - armor

### Dropping columns I don't intend to perform operations with

In [76]:
df.drop(columns = ['id', 'slug', 'artistName', 'image', 'imageGold', 'flavorText', 'cropImage'], inplace = True)

### Examining and addressing null values within individual columns

### Artist Name

In [67]:
df.artistName.value_counts(dropna = False)

James Ryman               53
Alex Horley Orlandelli    53
Matt Dixon                49
Ivan Fomin                48
Jim Nelson                47
                          ..
Michal Ivan                1
Scott Hampton              1
Patrik Hjelm               1
Derk Venneman              1
NaN                        1
Name: artistName, Length: 233, dtype: int64

In [68]:
df["artistName"].fillna("Unknown", inplace = True) 

In [69]:
df[df.artistName ==  'Unknown']

,id,collectible,slug,classId,multiClassIds,cardTypeId,cardSetId,rarityId,artistName,manaCost,...,flavorText,cropImage,duels,minionTypeId,health,attack,keywordIds,childIds,durability,armor
927,61967,1,61967-resizing-pouch,12,"[3, 2]",5,1466,1,Unknown,1,...,Just don't put a Resizing Pouch inside the Res...,https://d15f34w2p8l1cc.cloudfront.net/hearthst...,"{'relevant': True, 'constructed': True}",NaN,NaN,NaN,[21],NaN,NaN,NaN


### Text

In [62]:
df.text.value_counts()

<b>Taunt</b>                                                                                          15
<b>Charge</b>                                                                                          7
<b>Spell Damage +1</b>                                                                                 6
<b>Stealth</b>                                                                                         6
<b>Divine Shield</b>                                                                                   5
                                                                                                      ..
Transform all minions into random ones with the same Cost.                                             1
<b>Taunt</b> <b>Battlecry:</b> If you're holding a spell that costs (5) or more, restore 5 Health.     1
<b>Battlecry:</b> Your hero can't be targeted by spells or Hero Powers until your next turn.           1
<b>Discover</b> a Demon. Your next one costs (1) less. 

In [70]:
df["text"].fillna("No Effect", inplace = True) 

### Image Gold

In [71]:
df.imageGold.value_counts()

https://d15f34w2p8l1cc.cloudfront.net/hearthstone/7ffd053042135a148d4a6c13a66e87fe4c4d13d2fbffcc7bfb5f7216280f9e3b.png    1
https://d15f34w2p8l1cc.cloudfront.net/hearthstone/1f03720ef0563bdcd6cacc333f5a8f10b9347b010fd34f8776f25aa520743115.png    1
https://d15f34w2p8l1cc.cloudfront.net/hearthstone/c99786c4cbc3ba215aa8f66779156a5a801c4c17be8f61a69807857cc913bd98.png    1
https://d15f34w2p8l1cc.cloudfront.net/hearthstone/24d0d7987d78be6744f6ec894d8f727cfb3c2f4a587585d669119065b7651c0f.png    1
https://d15f34w2p8l1cc.cloudfront.net/hearthstone/946760437fb7215c9a707b8418c6174d0a3fedcf001b7b1b9d7ae816266223df.png    1
                                                                                                                         ..
https://d15f34w2p8l1cc.cloudfront.net/hearthstone/98a218d8a44e7961bb8cbc61397a444dc1fc27dc70dd9f139018622de8575c71.png    1
https://d15f34w2p8l1cc.cloudfront.net/hearthstone/0a8293607b5ac2d0a22e336e8b95d87a081dc3bc79e8b909e59c84d689414edc.png    1
https://